## Import Packages

In [0]:
%pip install requests

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
from pyspark.sql import functions as f 
import requests
import json
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

## Functions

In [0]:
def check_api_endpoint(url): 
  response = requests.get(url)
  if response.status_code == 200:
    return response
  else: 
    print(f"Failed to fetch from endpoint: {response.status_code}")

def get_meal_details(meal_id):
    """Fetch full details of a meal using its ID."""
    url = f"{base_url}/lookup.php?i={meal_id}"
    response = check_api_endpoint(url)
    meal_data = response.json().get("meals", [])[0]
    return meal_data

## Data Ingestions

#### Recipe API Data Ingestion
The following code pulls from the MealsDB API to gather a list of recipies available

Available Categories: ['Beef', 'Chicken', 'Dessert', 'Lamb', 'Miscellaneous', 'Pasta', 'Pork', 'Seafood', 'Side', 'Starter', 'Vegan', 'Vegetarian', 'Breakfast', 'Goat']


In [0]:
# API base URL
base_url = "https://www.themealdb.com/api/json/v1/1"

# get a list of categories
categories_url = f"{base_url}/categories.php"

response = check_api_endpoint(categories_url)

categories = response.json().get("categories", [])
category_list = [cat["strCategory"] for cat in categories]



# for each category, pull all the meals with their details into a nested list that will later become our dataframe
all_meal_data = []
for category in category_list: 
    url = f"{base_url}/filter.php?c={category}"
    response = check_api_endpoint(url)
    meals = response.json().get("meals", [])
    meal_ids = [meal["idMeal"] for meal in meals]

    for meal_id in meal_ids: 
        meal = get_meal_details(meal_id)

        ingredients = {
            f"Ingredient_{i+1}": meal[f"strIngredient{i+1}"] 
            for i in range(20) if meal[f"strIngredient{i+1}"]
        }

        all_meal_data.append((
            meal["idMeal"],
            meal["strMeal"],
            meal["strCategory"],
            meal["strArea"],
            meal["strInstructions"],
            meal["strMealThumb"],
            meal["strTags"],
            meal["strYoutube"],
            json.dumps(ingredients)
        ))

        
# define a schema for the Spark Dataframe
schema = StructType([
    StructField("idMeal", StringType(), True),
    StructField("strMeal", StringType(), True),
    StructField("strCategory", StringType(), True),
    StructField("strArea", StringType(), True),
    StructField("strInstructions", StringType(), True),
    StructField("strMealThumb", StringType(), True),
    StructField("strTags", StringType(), True),
    StructField("strYoutube", StringType(), True),
    StructField("ingredients", StringType(), True)
])

# create Spark dataframe
recipes_df = spark.createDataFrame(all_meal_data, schema)
recipes_df.display()
        
recipes_df.write.format("delta").mode("overwrite").saveAsTable("recipes")

In [0]:
%sql
-- see if the recipes table was made correctly 
select * from recipes

idMeal,strMeal,strCategory,strArea,strInstructions,strMealThumb,strTags,strYoutube,ingredients
52874,Beef and Mustard Pie,Beef,British,"Preheat the oven to 150C/300F/Gas 2. Toss the beef and flour together in a bowl with some salt and black pepper. Heat a large casserole until hot, add half of the rapeseed oil and enough of the beef to just cover the bottom of the casserole. Fry until browned on each side, then remove and set aside. Repeat with the remaining oil and beef. Return the beef to the pan, add the wine and cook until the volume of liquid has reduced by half, then add the stock, onion, carrots, thyme and mustard, and season well with salt and pepper. Cover with a lid and place in the oven for two hours. Remove from the oven, check the seasoning and set aside to cool. Remove the thyme. When the beef is cool and you're ready to assemble the pie, preheat the oven to 200C/400F/Gas 6. Transfer the beef to a pie dish, brush the rim with the beaten egg yolks and lay the pastry over the top. Brush the top of the pastry with more beaten egg. Trim the pastry so there is just enough excess to crimp the edges, then place in the oven and bake for 30 minutes, or until the pastry is golden-brown and cooked through. For the green beans, bring a saucepan of salted water to the boil, add the beans and cook for 4-5 minutes, or until just tender. Drain and toss with the butter, then season with black pepper. To serve, place a large spoonful of pie onto each plate with some green beans alongside.",https://www.themealdb.com/images/media/meals/sytuqu1511553755.jpg,"Meat,Pie",https://www.youtube.com/watch?v=nMyBC9staMU,"{""Ingredient_1"": ""Beef"", ""Ingredient_2"": ""Plain Flour"", ""Ingredient_3"": ""Rapeseed Oil"", ""Ingredient_4"": ""Red Wine"", ""Ingredient_5"": ""Beef Stock"", ""Ingredient_6"": ""Onion"", ""Ingredient_7"": ""Carrots"", ""Ingredient_8"": ""Thyme"", ""Ingredient_9"": ""Mustard"", ""Ingredient_10"": ""Egg Yolks"", ""Ingredient_11"": ""Puff Pastry"", ""Ingredient_12"": ""Green Beans"", ""Ingredient_13"": ""Butter"", ""Ingredient_14"": ""Salt"", ""Ingredient_15"": ""Pepper""}"
52878,Beef and Oyster pie,Beef,British,"Season the beef cubes with salt and black pepper. Heat a tablespoon of oil in the frying pan and fry the meat over a high heat. Do this in three batches so that you don’t overcrowd the pan, transferring the meat to a large flameproof casserole dish once it is browned all over. Add extra oil if the pan seems dry. In the same pan, add another tablespoon of oil and cook the shallots for 4-5 minutes, then add the garlic and fry for 30 seconds. Add the bacon and fry until slightly browned. Transfer the onion and bacon mixture to the casserole dish and add the herbs. Preheat the oven to 180C/350F/Gas 4. Pour the stout into the frying pan and bring to the boil, stirring to lift any stuck-on browned bits from the bottom of the pan. Pour the stout over the beef in the casserole dish and add the stock. Cover the casserole and place it in the oven for 1½-2 hours, or until the beef is tender and the sauce is reduced. Skim off any surface fat, taste and add salt and pepper if necessary, then stir in the cornflour paste. Put the casserole dish on the hob – don’t forget that it will be hot – and simmer for 1-2 minutes, stirring, until thickened. Leave to cool. Increase the oven to 200C/400F/Gas 6. To make the pastry, put the flour and salt in a very large bowl. Grate the butter and stir it into the flour in three batches. Gradually add 325ml/11fl oz cold water – you may not need it all – and stir with a round-bladed knife until the mixture just comes together. Knead the pastry lightly into a ball on a lightly floured surface and set aside 250g/9oz for the pie lid. Roll the rest of the pastry out until about 2cm/¾in larger than the dish you’re using. Line the dish with the pastry then pile in the filling, tucking the oysters in as well. Brush the edge of the pastry with beaten egg. Roll the remaining pastry until slightly large

## Data Scrubbing

## Model Tuning